In [73]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

link_ranking = 'https://www.icarros.com.br/opiniao-carros/ranking.jsf'
url_icarros = 'https://www.icarros.com.br'

In [74]:
def get_string_page(link, number_page):
    result = requests.post("https://www.icarros.com.br/opiniao-carros/ranking.jsf",
                    data={'j_idt107:j_idt116':'j_idt107:j_idt116','paginaAtual':str(number_page),'categoria':'0'})
    c = result.content
    soup = BeautifulSoup(c)
    
    return soup

def get_string_html(link):
    result = requests.get(link)
    c = result.content
    soup = BeautifulSoup(c)
    
    return soup

In [75]:
def get_links(page):
    links = []

    for sample in page:
        link = url_icarros + sample['href']

        links.append(link)
        
    return links

In [76]:
def get_score(table_score, string_html, link):
    marca = string_html.find("span", {"class": "makeName"}).get_text().replace("\n", "").lstrip(' ')
    modelo = string_html.find("span", {"class": "modelName"}).get_text().replace("\n", "").lstrip(' ')
    ano = string_html.find("span", {"class": "modelYear"}).get_text().replace("\n", "").lstrip(' ')
    price = string_html.find_all("span", {"class": "font-bold"})[0].text
    
    page_tag = string_html.find_all("ul", {"class": "new-pagination__container"})
    try:
        number_pages = len(page_tag[0].find_all('li', {"class": "new-pagination__item"})) - 1
    except:
        number_pages = 0

    positive_text = ""
    
    for i in range(1, number_pages + 1):
        link_temp = link
        link_temp = link_temp.replace('opinioes', '') + 'opiniao-do-dono?order=1&anoReview=3&pag='+ str(i) + '#opinioes'
        print (link_temp)
        
        textos_html = get_string_html(link_temp)
        textos_div = textos_html.find_all("div", {"class": "review-box__text"})

        for texto in textos_div:
            if 'Pontos positivos' in texto.text:
                positive_text = positive_text + texto.text

    score_list = []
    columns = ['Conforto / Acabamento', 'Consumo', 'Custo / Benefício', 'Design', 'Dirigibilidade', 'Manutenção', 'Performance']

    for score in table_score:
        score_list.append(score.text)

    df_score = pd.DataFrame([score_list], columns=columns)
    df_score['Marca'] = marca
    df_score['Modelo'] = modelo
    df_score['Ano'] = ano
    df_score['Pages'] = number_pages
    df_score['Texto Positivo'] = positive_text
    df_score['Price'] = price
    
    return df_score

In [77]:
def get_table_page(links):
    result = pd.DataFrame()

    for link in links:
        print (link)
        string_html = get_string_html(link)
        table_score = string_html.find_all("div", {"class": "font-lg font-bold"})
        df_score = get_score(table_score, string_html, link)

        result = pd.concat([df_score, result])
        
    return result

In [78]:
def get_link_img(string_html):
    imgs = string_html.find_all("img")

    for img in imgs:
        try:
            if 'galeriaimgmodelo' in img['data-src']:
                return img['data-src']
        except:
            pass

In [ ]:
def get_images_table():
    link_ranking = 'https://www.icarros.com.br/opiniao-carros/ranking.jsf'
    url_icarros = 'https://www.icarros.com.br'

    number_of_pages = 31

    result = pd.DataFrame()

    for i in range(number_of_pages):
        print ('Page:' + str(i))
        r = requests.post("https://www.icarros.com.br/opiniao-carros/ranking.jsf",
                    data={'j_idt107:j_idt116':'j_idt107:j_idt116','paginaAtual':str(i),'categoria':'0'})
        c = r.content
        soup = BeautifulSoup(c)

        samples = soup.find_all("a", {"class": "linkcatalogo"})
        links = get_links(samples)


        for link in links:
            string_html = get_string_html(link)
            table_score = string_html.find_all("div", {"class": "font-lg font-bold"})

            columns = ['Conforto / Acabamento', 'Consumo', 'Custo / Benefício', 'Design', 'Dirigibilidade', 'Manutenção', 'Performance']
            marca = string_html.find("span", {"class": "makeName"}).get_text().replace("\n", "").lstrip(' ')
            modelo = string_html.find("span", {"class": "modelName"}).get_text().replace("\n", "").lstrip(' ')
            link_img = get_link_img(string_html)

            print (link_img)

            df_score = pd.DataFrame([{'Marca': marca, 'Modelo':modelo, 'link': link_img}])

            result = pd.concat([df_score, result])
            
    return result

In [79]:
def main():
    link_ranking = 'https://www.icarros.com.br/opiniao-carros/ranking.jsf'
    url_icarros = 'https://www.icarros.com.br'
    
    number_of_pages = 5
    
    result = pd.DataFrame()
    
    for i in range(number_of_pages):
        print ('Page:' + str(i))
        r = requests.post("https://www.icarros.com.br/opiniao-carros/ranking.jsf",
                    data={'j_idt107:j_idt116':'j_idt107:j_idt116','paginaAtual':str(i),'categoria':'0'})
        c = r.content
        soup = BeautifulSoup(c)

        samples = soup.find_all("a", {"class": "linkcatalogo"})
        links = get_links(samples)
        
        try:
            result_page = get_table_page(links)

            result = pd.concat([result_page, result])
        except:
            pass

    #print (result)
    result['Marca'] = result['Marca'].str.upper()
    result['Modelo'] = result['Modelo'].str.upper()
    result = result[result['Price'].str.contains("R")]
    
    result['Price'] = result['Price'].str.replace('$', '').str.replace('R', '').str.lstrip().str.replace('.', '')
    result['Price'] = result['Price'].astype(float)
    
    columns = ['Conforto / Acabamento', 'Consumo', 'Custo / Benefício', 'Design', 'Dirigibilidade', 'Manutenção', 'Performance']

    for column in columns:
        result[column] = result[column].str.replace(',', '.').convert_objects(convert_numeric=True)
        
    return result
    

In [72]:
result = main()

Page:0


C:\Users\helder\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\helder\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


https://www.icarros.com.br/toyota/prius/opinioes
https://www.icarros.com.br/toyota/prius/opiniao-do-dono?order=1&anoReview=3&pag=1#opinioes
https://www.icarros.com.br/toyota/prius/opiniao-do-dono?order=1&anoReview=3&pag=2#opinioes
https://www.icarros.com.br/volkswagen/passat-cc/opinioes
https://www.icarros.com.br/volkswagen/passat-cc/opiniao-do-dono?order=1&anoReview=3&pag=1#opinioes
https://www.icarros.com.br/volkswagen/passat-cc/opiniao-do-dono?order=1&anoReview=3&pag=2#opinioes
https://www.icarros.com.br/suzuki/s-cross/opinioes
https://www.icarros.com.br/suzuki/s-cross/opiniao-do-dono?order=1&anoReview=3&pag=1#opinioes
https://www.icarros.com.br/suzuki/s-cross/opiniao-do-dono?order=1&anoReview=3&pag=2#opinioes
https://www.icarros.com.br/land-rover/defender/opinioes
https://www.icarros.com.br/land-rover/defender/opiniao-do-dono?order=1&anoReview=3&pag=1#opinioes
https://www.icarros.com.br/land-rover/defender/opiniao-do-dono?order=1&anoReview=3&pag=2#opinioes
https://www.icarros.com.b

https://www.icarros.com.br/renault/fluence/opiniao-do-dono?order=1&anoReview=3&pag=6#opinioes
https://www.icarros.com.br/renault/fluence/opiniao-do-dono?order=1&anoReview=3&pag=7#opinioes
https://www.icarros.com.br/audi/a3-sedan/opinioes
https://www.icarros.com.br/audi/a3-sedan/opiniao-do-dono?order=1&anoReview=3&pag=1#opinioes
https://www.icarros.com.br/audi/a3-sedan/opiniao-do-dono?order=1&anoReview=3&pag=2#opinioes
https://www.icarros.com.br/audi/a3-sedan/opiniao-do-dono?order=1&anoReview=3&pag=3#opinioes
https://www.icarros.com.br/audi/a3-sedan/opiniao-do-dono?order=1&anoReview=3&pag=4#opinioes
https://www.icarros.com.br/audi/a3-sedan/opiniao-do-dono?order=1&anoReview=3&pag=5#opinioes
https://www.icarros.com.br/mini/cooper/opinioes
https://www.icarros.com.br/mini/cooper/opiniao-do-dono?order=1&anoReview=3&pag=1#opinioes
https://www.icarros.com.br/mini/cooper/opiniao-do-dono?order=1&anoReview=3&pag=2#opinioes
https://www.icarros.com.br/bmw/x1/opinioes
https://www.icarros.com.br/bmw/

https://www.icarros.com.br/peugeot/308/opiniao-do-dono?order=1&anoReview=3&pag=5#opinioes
https://www.icarros.com.br/peugeot/308/opiniao-do-dono?order=1&anoReview=3&pag=6#opinioes
https://www.icarros.com.br/peugeot/308/opiniao-do-dono?order=1&anoReview=3&pag=7#opinioes
https://www.icarros.com.br/hyundai/hb20x/opinioes
https://www.icarros.com.br/hyundai/hb20x/opiniao-do-dono?order=1&anoReview=3&pag=1#opinioes
https://www.icarros.com.br/hyundai/hb20x/opiniao-do-dono?order=1&anoReview=3&pag=2#opinioes
https://www.icarros.com.br/hyundai/hb20x/opiniao-do-dono?order=1&anoReview=3&pag=3#opinioes
https://www.icarros.com.br/hyundai/hb20x/opiniao-do-dono?order=1&anoReview=3&pag=4#opinioes
https://www.icarros.com.br/hyundai/hb20x/opiniao-do-dono?order=1&anoReview=3&pag=5#opinioes
https://www.icarros.com.br/hyundai/hb20x/opiniao-do-dono?order=1&anoReview=3&pag=6#opinioes
https://www.icarros.com.br/fiat/500/opinioes
https://www.icarros.com.br/fiat/500/opiniao-do-dono?order=1&anoReview=3&pag=1#opini

C:\Users\helder\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [204]:
result

,Conforto / Acabamento,Consumo,Custo / Benefício,Design,Dirigibilidade,Manutenção,Performance,Marca,Modelo,Ano,Pages,Texto Positivo,Price
0,9.0,9.0,10.0,9.0,10.0,8.0,8.0,LADA,NIVA,1999,2,"\nPontos positivos: muito, muito mas muito bom...",10497.0
0,8.0,9.0,9.0,9.0,8.0,9.0,9.0,VOLKSWAGEN,KOMBI FURGÃO,2014,0,,35032.0
0,3.5,7.5,6.5,5.0,4.5,7.0,7.0,FIAT,UNO PICK-UP,1998,0,,9904.0
0,10.0,10.0,10.0,10.0,10.0,10.0,9.0,AUDI,Q7,2018,2,"\nPontos positivos: muito luxuoso, veloz e con...",416990.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FORD,RURAL,1982,0,,38800.0
0,3.0,9.0,9.0,3.0,7.0,8.0,7.0,FIAT,UNO FURGÃO,2016,3,\nPontos positivos: responde bem ao que se ped...,30252.0
0,5.8,8.2,7.3,6.8,7.3,7.8,7.2,VOLKSWAGEN,BRASÍLIA,1981,5,\nPontos positivos: todo bom\r\n ...,17800.0
0,9.0,9.0,9.5,10.0,10.0,9.0,10.0,VOLKSWAGEN,FUSCA,2016,7,\nPontos positivos: confiança no uso diário ou...,107352.0
0,5.0,0.0,5.0,10.0,7.0,8.0,7.0,FIAT,FIORINO,2019,4,\nPontos positivos: bom carro para trabalho co...,60070.0
0,6.5,8.5,9.0,6.2,9.0,9.5,9.2,FORD,COURIER,2013,5,\nPontos positivos: resist?ncia e performance ...,24940.0


In [128]:
columns = ['Conforto / Acabamento', 'Consumo', 'Custo / Benefício', 'Design', 'Dirigibilidade', 'Manutenção', 'Performance']

for column in columns:
    result[column] = result[column].str.replace(',', '.').convert_objects(convert_numeric=True)

C:\Users\helder\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.
C:\Users\helder\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [131]:
result

,Conforto / Acabamento,Consumo,Custo / Benefício,Design,Dirigibilidade,Manutenção,Performance,Marca,Modelo,Ano,Pages,Texto Positivo,Price
0,9.0,9.0,10.0,9.0,10.0,8.0,8.0,LADA,NIVA,1999,2,"\nPontos positivos: muito, muito mas muito bom...",10497.0
0,8.0,9.0,9.0,9.0,8.0,9.0,9.0,VOLKSWAGEN,KOMBI FURGÃO,2014,0,\nPontos positivos: motor do vw fox \r\n ...,35032.0
0,3.5,7.5,6.5,5.0,4.5,7.0,7.0,FIAT,UNO PICK-UP,1998,0,\nPontos positivos: 10\r\n ...,9904.0
0,10.0,10.0,10.0,10.0,10.0,10.0,9.0,AUDI,Q7,2018,2,"\nPontos positivos: muito luxuoso, veloz e con...",416990.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FORD,RURAL,1982,0,"\nPontos positivos: veiculo rústico, chama ate...",38800.0
0,3.0,9.0,9.0,3.0,7.0,8.0,7.0,FIAT,UNO FURGÃO,2016,3,\nPontos positivos: responde bem ao que se ped...,30252.0
0,5.8,8.2,7.3,6.8,7.3,7.8,7.2,VOLKSWAGEN,BRASÍLIA,1981,5,\nPontos positivos: todo bom\r\n ...,17800.0
0,9.0,9.0,9.5,10.0,10.0,9.0,10.0,VOLKSWAGEN,FUSCA,2016,7,\nPontos positivos: confiança no uso diário ou...,107352.0
0,5.0,0.0,5.0,10.0,7.0,8.0,7.0,FIAT,FIORINO,2019,4,\nPontos positivos: bom carro para trabalho co...,60070.0
0,6.5,8.5,9.0,6.2,9.0,9.5,9.2,FORD,COURIER,2013,5,\nPontos positivos: resist?ncia e performance ...,24940.0


In [139]:
link = 'https://www.icarros.com.br/land-rover/defender/opinioes'

In [151]:
link = link.replace('opinioes', '') + 'opiniao-do-dono?order=1&anoReview=3&pag='+ i + '#opinioes'

In [152]:
link

'https://www.icarros.com.br/nissan/altima/opiniao-do-dono?order=1&anoReview=3&pag=4#opinioes'

In [149]:
i='4'

In [150]:
link + 'opiniao-do-dono?order=1&anoReview=3&pag='+ i + '#opinioes'

'https://www.icarros.com.br/nissan/altima/opiniao-do-dono?order=1&anoReview=3&pag=4#opinioes'

In [205]:
result.to_csv('ranking.csv', index=False)

In [123]:
result

,Conforto / Acabamento,Consumo,Custo / Benefício,Design,Dirigibilidade,Manutenção,Performance,Marca,Modelo,Ano,Pages,Texto Positivo,Price
0,"9,0","9,0","10,0","9,0","10,0","8,0","8,0",LADA,NIVA,1999,2,"\nPontos positivos: muito, muito mas muito bom...",10497.0
0,"8,0","9,0","9,0","9,0","8,0","9,0","9,0",VOLKSWAGEN,KOMBI FURGÃO,2014,0,\nPontos positivos: motor do vw fox \r\n ...,35032.0
0,"3,5","7,5","6,5","5,0","4,5","7,0","7,0",FIAT,UNO PICK-UP,1998,0,\nPontos positivos: 10\r\n ...,9904.0
0,"10,0","10,0","10,0","10,0","10,0","10,0","9,0",AUDI,Q7,2018,2,"\nPontos positivos: muito luxuoso, veloz e con...",416990.0
0,,,,,,,,FORD,RURAL,1982,0,"\nPontos positivos: veiculo rústico, chama ate...",38800.0
0,"3,0","9,0","9,0","3,0","7,0","8,0","7,0",FIAT,UNO FURGÃO,2016,3,\nPontos positivos: responde bem ao que se ped...,30252.0
0,"5,8","8,2","7,3","6,8","7,3","7,8","7,2",VOLKSWAGEN,BRASÍLIA,1981,5,\nPontos positivos: todo bom\r\n ...,17800.0
0,"9,0","9,0","9,5","10,0","10,0","9,0","10,0",VOLKSWAGEN,FUSCA,2016,7,\nPontos positivos: confiança no uso diário ou...,107352.0
0,"5,0","0,0","5,0","10,0","7,0","8,0","7,0",FIAT,FIORINO,2019,4,\nPontos positivos: bom carro para trabalho co...,60070.0
0,"6,5","8,5","9,0","6,2","9,0","9,5","9,2",FORD,COURIER,2013,5,\nPontos positivos: resist?ncia e performance ...,24940.0


In [53]:
i = 0

In [54]:
r = requests.post("https://www.icarros.com.br/opiniao-carros/ranking.jsf",
            data={'j_idt107:j_idt116':'j_idt107:j_idt116','paginaAtual':str(i),'categoria':'0'})
c = r.content
soup = BeautifulSoup(c)

samples = soup.find_all("a", {"class": "linkcatalogo"})
links = get_links(samples)

C:\Users\helder\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\helder\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [65]:
link = links[4]

In [66]:
link

'https://www.icarros.com.br/nissan/altima/opinioes'

In [67]:
string_html = get_string_html(link)

C:\Users\helder\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\helder\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [72]:
test = string_html.find_all("span", {"class": "font-bold"})[0].text

In [73]:
test

'R$ 62.811'

In [ ]:
positive_text = ""

In [143]:
textos_div = string_html.find_all("div", {"class": "review-box__text"})

positive_text = ""

for texto in textos_div:
    if 'Pontos positivos' in texto.text:
        positive_text = positive_text + texto.text

In [144]:
positive_text

'\nPontos positivos: silencioso, fino acabamento que não poderia ser diferente de um toyota. educa vc a buscar mais e mais uma excelente condução.estou muito feliz com meu prius.\r\n                                                \nPontos positivos: comprei usado com 45.000km, está agora com 53.500km. um carro diferenciado pela inovação e pelo design. meu primeiro toyota, e agora entendi a fama: faz jus. super econômico, confortável, vários recursos tecnológicos, baixa manutenção, baixo custo de revisão, 8 anos de garantia do sistema híbrido. te educa a dirigir com com eficiência.\r\n                                                \nPontos positivos: ótimo projeto no geral, oferecendo silêncio incomparável durante a condução. ótima dirigibilidade, acabamento, consumo e opcionais.\r\n                                                \nPontos positivos: espaçoso, boa mala sem ser um carro grande. conforto e silêncio. economia nem se fala. dirigibilidade precisa e fácil. multimidia completa

In [251]:
link_ranking = 'https://www.icarros.com.br/opiniao-carros/ranking.jsf'
url_icarros = 'https://www.icarros.com.br'

number_of_pages = 2

result = pd.DataFrame()

for i in range(number_of_pages):
    print ('Page:' + str(i))
    r = requests.post("https://www.icarros.com.br/opiniao-carros/ranking.jsf",
                data={'j_idt107:j_idt116':'j_idt107:j_idt116','paginaAtual':str(i),'categoria':'0'})
    c = r.content
    soup = BeautifulSoup(c)

    samples = soup.find_all("a", {"class": "linkcatalogo"})
    links = get_links(samples)

    for link in links:
        print (link)
        string_html = get_string_html(link)
        table_score = string_html.find_all("div", {"class": "font-lg font-bold"})
        
        columns = ['Conforto / Acabamento', 'Consumo', 'Custo / Benefício', 'Design', 'Dirigibilidade', 'Manutenção', 'Performance']
        marca = string_html.find("span", {"class": "makeName"}).get_text().replace("\n", "").lstrip(' ')
        modelo = string_html.find("span", {"class": "modelName"}).get_text().replace("\n", "").lstrip(' ')
        
        df_score['Marca'] = marca
        df_score['Modelo'] = modelo

        result = pd.concat([df_score, result])

Page:0


C:\Users\helder\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\helder\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


https://www.icarros.com.br/toyota/prius/opinioes
https://www.icarros.com.br/volkswagen/passat-cc/opinioes
https://www.icarros.com.br/suzuki/s-cross/opinioes
https://www.icarros.com.br/land-rover/defender/opinioes
https://www.icarros.com.br/nissan/altima/opinioes
https://www.icarros.com.br/jac/t40/opinioes
https://www.icarros.com.br/renault/megane-cabriolet/opinioes
https://www.icarros.com.br/ssangyong/rexton/opinioes


KeyboardInterrupt: 

In [82]:
def get_images_table():
    link_ranking = 'https://www.icarros.com.br/opiniao-carros/ranking.jsf'
    url_icarros = 'https://www.icarros.com.br'

    number_of_pages = 31

    result = pd.DataFrame()

    for i in range(number_of_pages):
        print ('Page:' + str(i))
        r = requests.post("https://www.icarros.com.br/opiniao-carros/ranking.jsf",
                    data={'j_idt107:j_idt116':'j_idt107:j_idt116','paginaAtual':str(i),'categoria':'0'})
        c = r.content
        soup = BeautifulSoup(c)

        samples = soup.find_all("a", {"class": "linkcatalogo"})
        links = get_links(samples)


        for link in links:
            string_html = get_string_html(link)
            table_score = string_html.find_all("div", {"class": "font-lg font-bold"})

            columns = ['Conforto / Acabamento', 'Consumo', 'Custo / Benefício', 'Design', 'Dirigibilidade', 'Manutenção', 'Performance']
            marca = string_html.find("span", {"class": "makeName"}).get_text().replace("\n", "").lstrip(' ')
            modelo = string_html.find("span", {"class": "modelName"}).get_text().replace("\n", "").lstrip(' ')
            link_img = get_link_img(string_html)

            print (link_img)

            df_score = pd.DataFrame([{'Marca': marca, 'Modelo':modelo, 'link': link_img}])

            result = pd.concat([df_score, result])
            
    return result

Page:0


C:\Users\helder\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\helder\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


https://img2.icarros.com/dbimg/galeriaimgmodelo/1/125421_1.png
https://img1.icarros.com/dbimg/galeriaimgmodelo/1/4732_1.png
https://img2.icarros.com/dbimg/galeriaimgmodelo/1/132140_1.png
https://img0.icarros.com/dbimg/galeriaimgmodelo/1/8392_1.png
https://img2.icarros.com/dbimg/galeriaimgmodelo/1/20995_1.png
https://img1.icarros.com/dbimg/galeriaimgmodelo/1/131211_1.png
None
https://img1.icarros.com/dbimg/galeriaimgmodelo/1/27514_1.png
https://img0.icarros.com/dbimg/galeriaimgmodelo/1/24569_1.png
https://img2.icarros.com/dbimg/galeriaimgmodelo/1/123341_1.png
https://img1.icarros.com/dbimg/galeriaimgmodelo/1/113551_1.png
https://img2.icarros.com/dbimg/galeriaimgmodelo/1/98140_1.png
Page:1
https://img2.icarros.com/dbimg/galeriaimgmodelo/1/43912_1.png
https://img1.icarros.com/dbimg/galeriaimgmodelo/1/6190_1.png
https://img2.icarros.com/dbimg/galeriaimgmodelo/1/41995_1.png
https://img0.icarros.com/dbimg/galeriaimgmodelo/1/126766_1.png
https://img0.icarros.com/dbimg/galeriaimgmodelo/1/13181

https://img2.icarros.com/dbimg/galeriaimgmodelo/1/126005_1.png
https://img2.icarros.com/dbimg/galeriaimgmodelo/1/38450_1.png
https://img1.icarros.com/dbimg/galeriaimgmodelo/1/124282_1.png
None
https://img0.icarros.com/dbimg/galeriaimgmodelo/1/116476_1.png
https://img2.icarros.com/dbimg/galeriaimgmodelo/1/123947_1.png
None
https://img0.icarros.com/dbimg/galeriaimgmodelo/1/24407_1.png
None
https://img0.icarros.com/dbimg/galeriaimgmodelo/1/3096_1.png
https://img1.icarros.com/dbimg/galeriaimgmodelo/1/3388_1.png
Page:14
https://img0.icarros.com/dbimg/galeriaimgmodelo/1/131417_1.png
https://img2.icarros.com/dbimg/galeriaimgmodelo/1/7952_1.png
https://img2.icarros.com/dbimg/galeriaimgmodelo/1/125073_1.png
https://img1.icarros.com/dbimg/galeriaimgmodelo/1/121171_1.png
None
https://img1.icarros.com/dbimg/galeriaimgmodelo/1/132219_1.png
https://img2.icarros.com/dbimg/galeriaimgmodelo/1/7956_1.png
https://img0.icarros.com/dbimg/galeriaimgmodelo/1/12011_1.png
https://img2.icarros.com/dbimg/galeria

In [85]:
result.to_csv('link_image.csv', index=False)

In [86]:
result['link'].iloc[3]

'https://img1.icarros.com/dbimg/galeriaimgmodelo/1/120106_1.png'